In [1]:
import os, sys, random, math, time
import numpy as np
import torch
import matplotlib as plt
import torch.nn as nn
import torch_optimizer as optim2
sys.path.append('../../neural-sketch/src/')
sys.path.append('../../neural-sketch/')
sys.path.append('../../')
sys.path.append('../../../')
from src.models.frequency.countsketch import CountSketchQueryNetwork
from src.models.frequency.countmin import CountMinSketchQueryNetwork
from src.models.networks.sqnet import SketchNetwork, QueryNetworkTask
from src.models.networks.layers.residual import ResidualFC, ResidualFCwithPermute, ResidualFCLN
from src.models.networks.layers.permute import Permute
from src.datasets.zipf import get_zipf_probs, get_dataset_samples, ZipfOnlineDatasetLoader

In [2]:
class MeanDim(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        return torch.mean(x, dim=self.dim)
    

class ResNetMLPLN(torch.nn.Module):

    def __init__(self, in_dim, out_dim, hidden_dim=None, bottleneck_dim=None,num_layers=3, input_bn=False, output_bn=False):
        super().__init__()
        
        if hidden_dim == None:
            hidden_dim = in_dim*2
        residual_layers = [ResidualFCLN(hidden_dim, bottleneck_dim=bottleneck_dim) for _ in range(num_layers)]
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            *residual_layers,
            nn.Linear(hidden_dim, out_dim),
        )        
                
    def forward(self, x):
        y = self.net(x)
        return y

In [3]:


device = 'cuda:6'
datasets_path = ''
datasets_list = ['MNIST','fashion_MNIST','humans','dogs','KDD','heart','QuickDraw10','adults','bank'] 

def read_data(path):
    # Read data
    try:
        data = np.load(path)
        data_train = torch.from_numpy(data['train']).float()
        data_val = torch.from_numpy(data['val']).float()
        data_test = torch.from_numpy(data['test']).float()
    except:
        print('Error in reading data or data does not have the required fields. Inside the compressed file .npz it must be 3 numpy arrays with the following names: train, testa, and val')
        sys.exit(1)

    return data_train,data_val,data_test


In [4]:

def eval_network(encoder, decoder,data_val_loader):
    error = 0
    k = 0
    for i, (x,) in enumerate(data_val_loader):
        
        x = x.to(device)
        
        z =  encoder(x)
        query_output = torch.sigmoid(decoder(z))
        y_hat_t = (query_output > 0.5).float()
        error += torch.sum(torch.square(y_hat_t-x))
        k += x.shape[0]
    error = error / k
    return error




In [5]:


#

def train_eval(encoder, decoder, trainloader, valloader, output_path):

    param_list = [*encoder.parameters(), *decoder.parameters()]
    optimizer = optim2.RAdam(param_list, lr=0.0001, weight_decay=0.000001)

    criterion = nn.BCEWithLogitsLoss()
    best_mse = 9999
    patience = 0

    print('Start training!!!')
    num_epochs = 90000
    print('num epochs are', num_epochs)
    optimizer.zero_grad()


    for j in range(num_epochs):


        for i, (x,) in enumerate(trainloader):
            optimizer.zero_grad()
            x = x.to(device)

            z =  encoder(x)
            query_output = decoder(z)
            loss = criterion(query_output, x) 

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            if i % 100 == 0:
                print('Iter {} loss {}'.format(j,loss.item()))

            with torch.no_grad():
                if i % 1000 == 0:
                    error = eval_network(encoder, decoder,valloader)
                    
                    if error < best_mse:
                        best_mse = error
                        patience = 0
                        
                        torch.save(encoder.state_dict(), output_path+'_encoder.pt')
                        torch.save(decoder.state_dict(), output_path+'_decoder.pt')
                        
                        
                    else:
                        patience += 1
                    
                    print('Iter {} error {} - best: {}'.format(j,error,best_mse))
                    
                    if patience > 15:
                        print('finishing training!')
                        return 0
                    



In [6]:


for dataset_name in datasets_list:
    
    print('STARTING WITH ... ', dataset_name)
    path = datasets_path+dataset_name+'.npz'
    data_train,data_val,data_test = read_data(path)


    # load network
    sketch_size = 50
    N_DIMS = 1000
    encoder = ResNetMLPLN(N_DIMS, sketch_size, hidden_dim=4096, bottleneck_dim=4096, num_layers=5, output_bn=True).to(device)
    decoder = ResNetMLPLN(sketch_size, N_DIMS, hidden_dim=4096, bottleneck_dim=4096, num_layers=5, input_bn=True).to(device)


    train_dataset = torch.utils.data.TensorDataset(data_train)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)


    val_dataset = torch.utils.data.TensorDataset(data_val)
    valloader = torch.utils.data.DataLoader(val_dataset, batch_size=1024)

    output_path = ''

    train_eval(encoder, decoder, trainloader, valloader, output_path)

STARTING WITH ...  MNIST
Start training!!!
num epochs are 90000
Iter 0 loss 0.7012914419174194
Iter 0 error 495.0446472167969 - best: 495.0446472167969
Iter 1 loss 0.1133137196302414
Iter 1 error 44.51158142089844 - best: 44.51158142089844
Iter 2 loss 0.05436867102980614
Iter 2 error 20.939666748046875 - best: 20.939666748046875
Iter 3 loss 0.03786487132310867
Iter 3 error 14.15250015258789 - best: 14.15250015258789
Iter 4 loss 0.030277198180556297
Iter 4 error 11.256583213806152 - best: 11.256583213806152
Iter 5 loss 0.025513799861073494
Iter 5 error 9.766583442687988 - best: 9.766583442687988
Iter 6 loss 0.022400766611099243
Iter 6 error 8.563916206359863 - best: 8.563916206359863
Iter 7 loss 0.019867131486535072
Iter 7 error 7.939749717712402 - best: 7.939749717712402
Iter 8 loss 0.01771305501461029
Iter 8 error 7.302416801452637 - best: 7.302416801452637
Iter 9 loss 0.01642739586532116
Iter 9 error 6.708250045776367 - best: 6.708250045776367
Iter 10 loss 0.014673675410449505
Iter 1

In [7]:
print('done!!!!')

done!!!!
